# Краулеры

**План**

1. Что такое краулеры?
2. Как написать простой краулер?
3. Блокировки и способы их обхода

## Что такое краулеры?

Краулеры $-$ это боты / программы, которые "ползают" (*crawl*) по страницам сайта и собирают информацию. Все чаще использование таких программ запрещается правилами пользования сайтами, поэтому это формально нехорошо. Но так продолжают делать и это надо уметь. Запрещают по 2 основным причинам: не хотят делиться данными и боятся, что вы уроните сервер (если сайт $-$ маленький, а сервер $-$ не очень, то это довольно легко).

Поэтому нужно собирать данные аккуратно, чтобы:
1. вас не заблокировали по IP,
2. вы не навредили серверу.

## Как написать простой краулер?

In [1]:
import requests
from pprint import pprint

Вместо отдельных запросов лучше создать сессию, которая позволит хранить информацию между запросами и поддерживать то же соединение и не создавать каждый раз все заново, что влияет на производительность.

In [2]:
session = requests.session()

Попробуем сделать запрос, просто вместо requests.get мы пишет session.get.

In [3]:
response = session.get('https://ru.wikipedia.org')

Может найти значения нашего IP-адреса

In [4]:
response.headers['X-Client-IP']

'34.121.176.38'

Можно ли нас отследить по IP? С определенной точностью, можно узнать округ или компанию, к которой привязан любой IP. Пример сервиса, который позволяет это сделать [здесь](https://whatismyipaddress.com/ip-lookup).

Посмотреть на `headers` запроса

In [5]:
pprint(dict(response.headers))

{'accept-ch': '',
 'accept-ranges': 'bytes',
 'age': '1842',
 'cache-control': 'private, s-maxage=0, max-age=0, must-revalidate, '
                  'no-transform',
 'content-encoding': 'gzip',
 'content-language': 'ru',
 'content-length': '30320',
 'content-type': 'text/html; charset=UTF-8',
 'date': 'Tue, 12 Nov 2024 08:01:08 GMT',
 'last-modified': 'Tue, 12 Nov 2024 08:00:35 GMT',
 'nel': '{ "report_to": "wm_nel", "max_age": 604800, "failure_fraction": 0.05, '
        '"success_fraction": 0.0}',
 'report-to': '{ "group": "wm_nel", "max_age": 604800, "endpoints": [{ "url": '
              '"https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" '
              '}] }',
 'server': 'mw-web.codfw.main-588896774d-xknkh',
 'server-timing': 'cache;desc="hit-front", host;desc="cp2039"',
 'set-cookie': 'WMF-DP=23c;Path=/;HttpOnly;secure;Expires=Tue, 12 Nov 2024 '
               '00:00:00 GMT',
 'strict-transport-s

### Стратегии сбора данных


По сути краулеры выполняют сбор страниц (их HTML) как мы это делали на прошлом занятии, но делают они это циклами (или циклами циклов). Можно выделить разные стратегии сбора данных:
    
**По типу навигации**

1. Все страницы со ссылками имеют удобные номера ("https://ficbook.net/fanfiction/no_fandom/originals?p=2"), обычно просто `p=<число>` или `page=<число>`. В этом случае вам нужно просто подставлять цифры подробнее про параметры передаваемые в ссылке можно посмотреть [здесь](https://en.wikipedia.org/wiki/Query_string).
2. Страницы называются как-то не структурированно (например, по названиям блоков). Тут нужно собирать ссылки на эти страницы и потом по ним ходить и собирать конечные странички.
3. Все расположено на одной страничке и догружается с использованием [WebSocket](https://en.wikipedia.org/wiki/WebSocket) или других технологий, при адрес в адресной строке никак не изменяется, данные могут догружаться на сайт автоматически по мере скролла страницы.

**По скорости обновления**

1. Если сайт довольно статичный по контенту (медленно появляются и удаляются материалы), то можно сначал собрать ссылки, а потом по ним ходить.
2. Если сайт очень динамичный по контенту (например, объявления на крупном сайте), вам нужно при получении страничкии ссылок сразу их обходить, а потом переходить к следующей, потому что ко времени получения исчерпывающего списка ссылок по сайту многие будут уже удалены или недоступны.



## Блокировки и способы их обхода

Для того, чтобы предотвратить автоматический сбор информации с некого сайта, применяются различные инструменты, которые определяют роботов и блокируют запросы с адресов, которые были классифицированы как роботы. Чтобы не заблокировали домашний/учебный IP, лучше сразу задуматься об этих мерах и предотвратить возможные проблемы. Кстати, Википедия не блокирует и можно спокойно скачивать без каких-либо проблем.

Чтобы их обойти, можно попробовать несколько инструментов:
1. `time.sleep(x)` - задержка между запросами, чтобы слишком большая скорость запросов не показалась подозрительной или ваши запросы не уронили сервер небольшого ресурса (например, региональной газеты).
2. `time.sleep(<случайный промежуток времени>)` - это более хитрая версия, когда время задержки - это случайное число из некоторого отрезка (модуль `random`).
3. изобразить браузер - при запросе отправляется информация о том, из какого приложения пришел запрос (например, Google Chrome), запросы сделанные из браузера больше похожи на человеческие, для этого нужно задать `user-agent` в параметрах (а его выбирать случайно с помощью `fake_useragent`).
4. использовать прокси - существуют ресурсы с бесплатными списками открытых прокси, через которые можно пропускать ваш запрос и сервер будет думать, что запросы приходят из разных мест (anonymous и elite классы прокси) или использовать анонимизированные сети к примеру сеть [Tor](https://en.wikipedia.org/wiki/Tor_(network)) и аналоги.

### Пауза между запросами

In [6]:
import time
from datetime import datetime

In [7]:
for _ in range(5):
    response = session.get('https://ru.wikipedia.org')
    print(datetime.now())
    time.sleep(1)

2024-11-12 08:35:06.562489
2024-11-12 08:35:07.629049
2024-11-12 08:35:08.694599
2024-11-12 08:35:09.759963
2024-11-12 08:35:10.825281


### Притвориться нормальным браузером

In [11]:
# pip install fake_useragent

In [10]:
from fake_useragent import UserAgent

Можно настроить так, чтобы не проверять безопасность соединения, что иногда вызывает ошибки. Но это можно делать с сайтмаи, которым вы доверяете.

In [12]:
ua = UserAgent()

In [13]:
headers = {'User-Agent': ua.random}
print(headers)
response = session.get('https://ru.wikipedia.org', headers=headers)

{'User-Agent': 'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Mobile Safari/537.36'}


### Пауза между запросами (случайное время)

In [14]:
import random

`random.uniform` позволяет получить случайное число из отрезка.

In [15]:
random.uniform(1, 3)

2.1778689099826725

In [16]:
for _ in range(5):
    response = session.get('https://ru.wikipedia.org')
    print(datetime.now())
    time.sleep(random.uniform(1.1, 5.2))

2024-11-12 08:35:56.708548
2024-11-12 08:36:00.878587
2024-11-12 08:36:06.009044
2024-11-12 08:36:07.483674
2024-11-12 08:36:10.206681


### Подключение через прокси

Прокси-сервер — это дополнительное звено между вами и интернетом, через него пойдет подключение и сайт не будет знать, что это вы посылаете запрос.

1. Адреса прокси можно взять со специальных сайтов: например, [https://hideip.me/ru/proxy/httplist](https://hideip.me/ru/proxy/httplist).
2. Потом проверить, что они рабочие, прежде чем использовать: [https://checkerproxy.net/](https://checkerproxy.net/).

In [17]:
proxy = {"https://": "https://176.213.141.107:8080"}

response = session.get('http://ru.wikipedia.org', proxies=proxy)
print(response.headers['X-Client-IP'])

34.121.176.38


## Пример

Давайте обкачаем немного новостей с сайта вышки.

1. Страницы имеют вид "https://www.hse.ru/news/page1.html", поэтому можно просто идти циклом.
2. Достанем дату публикации, заголовок, краткое описание (из станицы со списком новостей), текст полной статьи и метки (из самой страницы новости)
3. Сохраним в датафрейм.

In [18]:
from bs4 import BeautifulSoup
import re

### **Шаг 1. Найти страницы**

Посмотрим, как устроены новости и скачаем одну страницу:

In [19]:
page_number = 2
url = f'https://www.hse.ru/news/page{page_number}.html'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

Распарсим с помощью `BeautifulSoup`:

In [20]:
soup = BeautifulSoup(page, 'html.parser')

Найдем отдельные посты:

In [21]:
news = soup.find_all('div', {'class': 'post'})

In [22]:
len(news)

10

Найдем заголовок-ссылку и запомним текст заголовка:

In [23]:
title_obj = news[0].find('a')
title_obj

<a class="link link_dark2 no-visited" href="/news/edu/984409969.html">ИНФОТЕХ-2024: «понять перспективы и ограничения использования ИИ в образовании»</a>

In [24]:
title = title_obj.text
title

'ИНФОТЕХ-2024: «понять перспективы и ограничения использования ИИ в образовании»'

Достанем свойства этой ссылки (куда ведет, `class`):

In [25]:
attrs = title_obj.attrs
attrs

{'href': '/news/edu/984409969.html',
 'class': ['link', 'link_dark2', 'no-visited']}

Достанем саму ссылку:

In [26]:
href = title_obj.attrs['href']
href

'/news/edu/984409969.html'

Достанем текст новости:

In [27]:
short_text = news[0].find('div', {'class': 'post__text'}).text
short_text

'В конце октября в рамках XVII Тюменского цифрового форума информационных технологий «ИНФОТЕХ-2024» прошел круглый стол «Эксперименты с ИИ в образовании». Эксперты Высшей школы экономики, Московского городского педагогического университета, Уральского федерального университета и Тюменского государственного университета обсудили практический опыт разработки и внедрения технологий ИИ в образовательный процесс, обозначили основные вызовы, связанные с быстрым развитием образовательных решений на базе ИИ.'

Достанем день, месяц, год публикации:

In [28]:
pub_day = news[0].find('div', {'class': 'post-meta__day'}).text
pub_day

'8'

In [29]:
pub_month = news[0].find('div', {'class': 'post-meta__month'}).text
pub_month

'ноя'

In [30]:
pub_year = news[0].find('div', {'class': 'post-meta__year'}).text
pub_year

'2024'

### **Шаг 2. Научиться парсить страничку самой новости**

Возьмем ссылку на полную новость и соединим с адерсом сайта (т.к. ссылка относительная):

In [31]:
url_o = "https://www.hse.ru" + href
url_o

'https://www.hse.ru/news/edu/984409969.html'

Скачаем ее и распарсим:

In [32]:
req = session.get(url_o, headers={'User-Agent': ua.random})
page = req.text

soup = BeautifulSoup(page, 'html.parser')

Сохраним текст, распечатаем кусочек:

In [33]:
full_text = soup.find('div', {'class': 'post__content'}).text
full_text[:200]

'ИНФОТЕХ-2024: «понять перспективы и ограничения использования ИИ в образовании»© Высшая школа экономикиВ конце октября в рамках XVII Тюменского цифрового форума информационных технологий «ИНФОТЕХ-2024'

Найдем теги, которые присвоены статье:

In [34]:
meta = soup.find('div', {'class': 'articleMeta'})

tags = meta.find_all('a', {'class': 'tag'})
tags = [t.text for t in tags]
tags

['искусственный интеллект']

### **Шаг 3. Оформляем нормально в функции**

Сделаем словарь соответствий имени месяца и его номера:

In [35]:
months = {
    value: key + 1
    for key, value in enumerate(
        ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    )
}

Парсим информацию из страницы со списком новостей (блок одной новости):

In [36]:
def parse_news_page_block(one_block):
    block = {}
    a = one_block.find('a')
    block['title'] = a.text
    block['href'] = a.attrs['href']
    block['short_text'] = one_block.find('div', {'class': 'post__text'}).text
    block['pub_day'] = int(one_block.find('div', {'class': 'post-meta__day'}).text)
    block['pub_month'] = months[one_block.find('div', {'class': 'post-meta__month'}).text]
    block['pub_year'] = int(one_block.find('div', {'class': 'post-meta__year'}).text)
    return block

Парсим отдельную страницу новости:

In [37]:
def parse_one_article(block):
    url_one = 'http://www.hse.ru' + block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    block['full_text'] = soup.find('div', {'class': 'post__content'}).text
    meta = soup.find('div', {'class': 'articleMeta'})
    tags = meta.find_all('a', {'class': 'tag'})
    block['tags'] = [t.text for t in tags]
    return block

Регулярное выражение для того, чтобы достать ID новости и не повторяться:

In [38]:
regex_hse_id = re.compile('/([0-9]*?).html')

Обработать N-ую страницу новостей:

In [39]:
def get_nth_page(page_number):
    # скачиваем
    url = f'https://www.hse.ru/news/page{page_number}.html'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    # находим новости
    news = soup.find_all('div', {'class': 'post'})

    # идем по новостям и обрабатываем их
    blocks = []
    for n in news:
        try:
            blocks.append(parse_news_page_block(n))
        except Exception as e:
            print(e)

    # идем по отдельным статьям и достаем информацию
    result = []
    for b in blocks:
        if b['href'].startswith('/'):
            idx = regex_hse_id.findall(b['href'])[0]
            try:
                res = parse_one_article(b)
                res['hse_id'] = idx
                result.append(res)
            except Exception as e:
                print(e)

    # возвращаем найденную информацию
    return result

### **Шаг 4. Сохраняем в датафрейм**


In [40]:
from tqdm.auto import tqdm
import pandas as pd

Напишем функцию, куда передаем количество страниц и она выполняет все нужные действия:

In [41]:
def run_all(n_pages):
    blocks = []
    for i in tqdm(range(n_pages)):
        blocks.extend(get_nth_page(i+1))

    return blocks

Запускаем на 20 первых страниц:

In [42]:
blocks = run_all(20)

df = pd.DataFrame(blocks)

  0%|          | 0/20 [00:00<?, ?it/s]

Посмотрим на 10 самых популярных тегов

In [43]:
from collections import Counter

In [44]:
tags = [tag for tags in df["tags"] for tag in tags]

In [45]:
for title, counts in sorted(Counter(tags).items(), key=lambda x: -x[-1])[:10]:
    print(counts,"\t", title)

41 	 репортаж о событии
30 	 исследования и аналитика
21 	 искусственный интеллект
18 	 новое в ВШЭ
17 	 достижения
17 	 студенты
16 	 международное сотрудничество
14 	 приглашение к участию
14 	 довузовская подготовка
11 	 олимпиады Вышки


Посмотрим, сколько публикаций по месяцам

In [46]:
df.groupby("pub_month")["hse_id"].count()

,hse_id
pub_month,
9,52
10,84
11,31


## Задание на семинар

Сделайте краулер фильмов аналогичный тому, что выше.

У вас есть список на [Letterboxd](https://letterboxd.com/) c перечислением всех фильмов из книжки *1001 Movies You Must See Before You Die*: https://letterboxd.com/peterstanley/list/1001-movies-you-must-see-before-you-die/detail/.

1. Все страницы выглядят как `https://letterboxd.com/peterstanley/list/1001-movies-you-must-see-before-you-die/detail/page/<номер>/`, поэтому снова можно идти циклом.
2. Достанем номер фильма, его название, год выпуска со страницы списка.
3. Достанем режиссера, слоган (если есть) и краткое описание с самой страницы фильма.
4. Сохраним в датафрейм.